# Initial Settings

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from Assets import jalali
import statistics
import plotly.graph_objects as go
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import voronoi_diagram as svd
from shapely.wkt import loads as load_wkt
from datetime import datetime
from tabulate import tabulate

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)



# Modify Date And Water Table
# Date Type: 
#   1.Gregorian (Text): 2000-11-1
#   2.Persian (Text): 1399-1-1
# Date & Value:
#   Column of Pandas Dataframe
def convert_to_day_15(data, date_type="persian"):
    
    data = data.reset_index(drop=True)
            
    df = data[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"]]
    
    if date_type == "gregorian":
        df["DATE_GREGORIAN"] = data.DATE_GREGORIAN_RAW.apply(pd.to_datetime)
        df["DATE_PERSIAN"] = list(
            map(
                lambda i: jalali.Gregorian(i.date()).persian_string(),
                df["DATE_GREGORIAN"]
            )
        )
        
    elif date_type == "persian":
        df["DATE_PERSIAN"] = data.DATE_PERSIAN_RAW
        df["DATE_GREGORIAN"] = list(
            map(
                lambda i: jalali.Persian(i).gregorian_string(),
                df["DATE_PERSIAN"]
            )
        )
        df["DATE_GREGORIAN"] = df["DATE_GREGORIAN"].apply(pd.to_datetime)
        
    else:
        pass
    
    df["VALUE"] = data.WATER_TABLE_RAW 
           
    df["DELTA_DAY"] = df["DATE_GREGORIAN"].diff().dt.days
    
    df["DATE_PERSIAN_NEW"] = list(
        map(
            lambda i: f"{int(i.split('-')[0])}-{int(i.split('-')[1])}-{15}",
            df["DATE_PERSIAN"]
        )
    )
    
    df["DATE_GREGORIAN_NEW"] = list(
        map(
            lambda i: jalali.Persian(i).gregorian_string(),
            df["DATE_PERSIAN_NEW"]
        )
    )
    
    df["DATE_GREGORIAN_NEW"] = df["DATE_GREGORIAN_NEW"].apply(pd.to_datetime)
    
    df["VALUE_NEW"] = df["VALUE"]
    
    A = []
    
    A.append(df["VALUE"][0])
    
    for i in range(1, len(df) - 1):
        if int(df["DATE_PERSIAN"][i].split('-')[2]) >= 15:
            NEW_VALUE = df["VALUE"][i-1] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i-1]).days) / ((df["DATE_GREGORIAN"][i] - df["DATE_GREGORIAN"][i-1]).days)) * (df["VALUE"][i] - df["VALUE"][i-1]))
            A.append(NEW_VALUE)
        else:
            NEW_VALUE = df["VALUE"][i] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i]).days) / ((df["DATE_GREGORIAN"][i+1] - df["DATE_GREGORIAN"][i]).days)) * (df["VALUE"][i+1] - df["VALUE"][i]))
            A.append(NEW_VALUE)
    
    A.append(df["VALUE"][len(df) - 1])
            
    df["VALUE_NEW"] = A
        
    return df


def create_date_day15(min, max):
    """[summary]

    Args:
        min ([type]): [description]
        max ([type]): [description]

    Returns:
        [type]: [description]
    """
    result = []
    min_list = list(map(lambda x: int(x), min.split("-")))
    max_list = list(map(lambda x: int(x), max.split("-")))
    for y in range(min_list[0], max_list[0] + 1):
        for m in range(1, 13):
            result.append(f"{y}-{m}-15")

    result = pd.DataFrame(
        {"DATE_PERSIAN" : result}
    )
    result['DATE_GREGORIAN'] = result.apply(
        lambda x: jalali.Persian(x["DATE_PERSIAN"]).gregorian_string(), 
        axis=1
    )
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)
    result = result[result["DATE_GREGORIAN"] >= pd.to_datetime(jalali.Persian(min).gregorian_string())]
    result = result[result["DATE_GREGORIAN"] <= pd.to_datetime(jalali.Persian(max).gregorian_string())]
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)  
    return result


def dropHolesBase(plg):

	'''

	BASIC FUNCTION TO REMOVE / DROP / FILL THE HOLES.

	PARAMETERS:

		plg: plg WHO HAS HOLES / EMPTIES.
			Type: shapely.geometry.MultiPolygon OR shapely.geometry.Polygon

	RETURNS:
		A shapely.geometry.MultiPolygon OR shapely.geometry.Polygon object
	

	'''

	if isinstance(plg, MultiPolygon):

		return MultiPolygon(Polygon(p.exterior) for p in plg)

	elif isinstance(plg, Polygon):

		return Polygon(plg.exterior)


def dropHoles(gdf):

	'''

	REMOVE / DROP / FILL THE HOLES / EMPTIES FOR ITERMS IN GeoDataFrame.
	
	PARAMETERS:
		gdf:
			Type: geopandas.GeoDataFrame

	RETURNS:
		gdf_nohole: GeoDataFrame WITHOUT HOLES
			Type: geopandas.GeoDataFrame
	
	'''

	gdf_nohole = gpd.GeoDataFrame()

	for g in gdf['geometry']:

		geo = gpd.GeoDataFrame(geometry=gpd.GeoSeries(dropHolesBase(g)))

		gdf_nohole=gdf_nohole.append(geo,ignore_index=True)

	gdf_nohole.rename(columns={gdf_nohole.columns[0]:'geometry'}, inplace=True)

	gdf_nohole.crs = gdf.crs

	gdf.rename(columns={'geometry': 'geometry_old'}, inplace=True)

	gdf["geometry_new"] = gdf_nohole

	gdf.rename(columns={'geometry_new': 'geometry'}, inplace=True)

	gdf.drop(['geometry_old'], axis=1, inplace=True)

	return gdf


def thiessen_polygons(gdf, mask):

	'''

	CREATE VORONOI DIAGRAM / THIESSEN POLYGONS:

	PARAMETERS:

		gdf: POINTS / POLYGONS TO BE USED TO CREATE VORONOI DIAGRAM / THIESSEN POLYGONS.
            Type: geopandas.GeoDataFrame

		mask: POLYGON VECTOR USED TO CLIP THE CREATED VORONOI DIAGRAM / THIESSEN POLYGONS.
			Type: GeoDataFrame, GeoSeries, (Multi)Polygon

	RETURNS:

		gdf_vd: THIESSEN POLYGONS
			Type: geopandas.geodataframe.GeoDataFrame
	
	'''
	
	gdf.reset_index(drop=True)

	# CONVERT TO shapely.geometry.MultiPolygon
	smp = gdf.unary_union

	# CREATE PRIMARY VORONOI DIAGRAM BY INVOKING shapely.ops.voronoi_diagram
	poly = load_wkt('POLYGON ((42 24, 64 24, 64 42, 42 42, 42 24))')
	smp_vd = svd(smp, envelope=poly)

	# CONVERT TO GeoSeries AND explode TO SINGLE POLYGONS
	gs = gpd.GeoSeries([smp_vd]).explode()

	# CONVERT TO GEODATAFRAME
	# NOTE THAT IF GDF WAS shapely.geometry.MultiPolygon, IT HAS NO ATTRIBUTE 'crs'
	gdf_vd_primary = gpd.geodataframe.GeoDataFrame(geometry=gs, crs=gdf.crs)
	
	# RESET INDEX
	gdf_vd_primary.reset_index(drop=True)
	
	# SPATIAL JOIN BY INTERSECTING AND DISSOLVE BY `index_right`
	gdf_temp = (gpd.sjoin(gdf_vd_primary, gdf, how='inner', op='intersects').dissolve(by='index_right').reset_index(drop=True))

	gdf_vd = gpd.clip(gdf_temp, mask)

	gdf_vd = dropHoles(gdf_vd)

	return gdf_vd

# Read Data

## Read Data And GeoInfo

In [ ]:
xls = pd.ExcelFile('Database/HydrographData.xlsx')
Data = pd.read_excel(xls, sheet_name='Data')
GeoInfo = pd.read_excel(xls, sheet_name='GeoInfo')

# GeoInfo
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.rstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.lstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.replace('ي','ی'))
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.replace('ئ','ی'))
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

# Data
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
Data[COLs] = Data[COLs].apply(lambda x: x.str.rstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.lstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.replace('ي','ی'))
Data[COLs] = Data[COLs].apply(lambda x: x.str.replace('ئ','ی'))
Data[COLs] = Data[COLs].apply(lambda x: x.str.replace('ك', 'ک'))


In [ ]:
Data

In [ ]:
GeoInfo

## LOAD SHAPEFILES

In [ ]:
gdf = gpd.read_file("GeoDatabase/Wells_Selected.geojson")
gdf = gdf.set_crs("EPSG:32640", allow_override=True)
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ي','ی'))
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ئ','ی'))
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ك', 'ک'))



mask = gpd.read_file("GeoDatabase/Aquifers_Selected.geojson")
mask = mask.set_crs("EPSG:32640", allow_override=True)
COLs = ['AQ_NAME', 'MA_NAME']
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ي','ی'))
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ئ','ی'))
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ك', 'ک'))


In [ ]:
ax = mask.plot(color='white', edgecolor='black', figsize=(10,10))
gdf.plot(ax=ax, marker='o', color='red', markersize=20)

# Convert Date

In [ ]:
Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)

Data['DATE_CHECK'] = np.where(
    Data["DATE_PERSIAN_RAW"].isna(),
    np.where(
        Data["DATE_GREGORIAN_RAW"].isna(),
        np.NaN,
        "G"
    ),
    "P"  
)

Data['DATE_PERSIAN_RAW'] = Data.apply(
    lambda x: jalali.Gregorian(x["DATE_GREGORIAN_RAW"].date()).persian_string() if x["DATE_CHECK"] == "G" else x["DATE_PERSIAN_RAW"], 
    axis=1
)

Data['DATE_GREGORIAN_RAW'] = Data.apply(
    lambda x: jalali.Persian(x["DATE_PERSIAN_RAW"]).gregorian_string() if x["DATE_CHECK"] == "P" else x["DATE_GREGORIAN_RAW"], 
    axis=1
)

Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)


Data.drop(['DATE_CHECK'], axis=1, inplace=True)

Data.sort_values(
    by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"], 
    inplace=True
)

In [ ]:
Data

# Remove Duplicate Rows

In [ ]:
Data = Data.drop_duplicates(
    subset=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_GREGORIAN_RAW'],
    keep='last'
)

In [ ]:
Data

# Gap Filling

## Convert To Day 15

In [ ]:
# Remove NanN Data In Column "WATER_TABLE_RAW"
Data.dropna(
    subset=["WATER_TABLE_RAW"],
    inplace=True
)

Data.reset_index(
    drop=True,
    inplace=True
)

wt_date_converted = Data.groupby(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"])\
    .apply(convert_to_day_15)\
        .reset_index(drop=True)[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN", "DATE_PERSIAN_NEW", "DATE_GREGORIAN", "DATE_GREGORIAN_NEW", "VALUE_NEW"]]

wt_date_converted.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_PERSIAN", "DATE_GREGORIAN_RAW","DATE_GREGORIAN", "WATER_TABLE"]

Data = Data.merge(
    right=wt_date_converted,
    how="left",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_GREGORIAN_RAW"]
)

In [ ]:
Data

# Remove Duplicate Rows

In [ ]:
Data = Data.drop_duplicates(
    subset=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_GREGORIAN'],
    keep='last'
)

Data

## Interpolate

In [ ]:
tmp = pd.DataFrame()

for mn in list(Data["MAHDOUDE_NAME"].unique()):
    df_mn = Data[(Data["MAHDOUDE_NAME"] == mn)]
    
    for an in list(df_mn["AQUIFER_NAME"].unique()):
        df_an = df_mn[(df_mn["AQUIFER_NAME"] == an)]
        
        for ln in list(df_an["LOCATION_NAME"].unique()):
            df = df_an[(df_an["LOCATION_NAME"] == ln)]
            
            date = create_date_day15(
                min = df.DATE_PERSIAN.min(),
                max = df.DATE_PERSIAN.max()
            )
            
            df = date.merge(
                df,
                how="left",
                on=["DATE_PERSIAN", "DATE_GREGORIAN"]
            )
            
            df["WATER_TABLE_INTERPOLATE"] = df["WATER_TABLE"].interpolate(method='akima')
            # df["WATER_TABLE_LINEAR"] = df["WATER_TABLE_LINEAR"].interpolate(method='ffill', limit=3)
            # df["WATER_TABLE_LINEAR"] = df["WATER_TABLE_LINEAR"].interpolate(method='bfill', limit=3)
            # df["WATER_TABLE_SPLINE"] = df["WATER_TABLE"].interpolate(method='akima')
            # df["WATER_TABLE_SPLINE"] = df["WATER_TABLE_SPLINE"].interpolate(method='ffill', limit=3)
            # df["WATER_TABLE_SPLINE"] = df["WATER_TABLE_SPLINE"].interpolate(method='bfill', limit=3)
            df["MAHDOUDE_NAME"] = mn
            df["AQUIFER_NAME"] = an
            df["LOCATION_NAME"] = ln
            df["DATA_STATE"] = df["DATA_STATE"].fillna("M")
            df["STORAGE_COEFFICIENT_LOCATION"] = df["STORAGE_COEFFICIENT_LOCATION"].unique()[0]
            
            df = df[[
                "MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME",
                "DATE_GREGORIAN", "DATE_PERSIAN",
                "WATER_TABLE", "WATER_TABLE_INTERPOLATE", "STORAGE_COEFFICIENT_LOCATION", "THISSEN_LOCATION", "THISSEN_AQUIFER",
                "DATA_STATE", "DATE_GREGORIAN_RAW", "DATE_PERSIAN_RAW", "WATER_TABLE_RAW"	
            ]]

            tmp = pd.concat([tmp, df], axis=0)

Data = tmp.copy().reset_index(drop=True)

del tmp

In [ ]:
Data

# Select Well

In [ ]:
# from random import randint
# Data = Data[Data["LOCATION_NAME"] != Data["LOCATION_NAME"].unique()[randint(0,9)]]

# Thissen Calculation

## Initial Settings

In [ ]:
# SELECT LEVEL
SELECTED_LEVEL = 'LEVEL_SRTM'

# FILTER DATE
Data = Data[Data["DATE_GREGORIAN"] >= datetime.strptime("2001-10-01", '%Y-%m-%d')]

# CREATE FUNCTIONs
def f (df_raw):
    df = df_raw.reset_index(drop=True).dropna(subset=["WATER_TABLE"])
    vd_result = gpd.GeoDataFrame()    
    gdf_d = gdf[gdf["LOCATION_NAME"].isin(df["LOCATION_NAME"])]
    # TODO: Find Better Way
    for i in range(len(mask)):
        mask_tmp = mask.iloc[[i]].reset_index(drop=True)
        gdf_d['CHECK'] = gdf_d['geometry'].apply(lambda x: mask_tmp.contains(x))
        gdf_tmp = gdf_d[gdf_d['CHECK']].reset_index(drop=True)
        if len(gdf_tmp) > 0:     
            vd = thiessen_polygons(gdf_tmp, mask_tmp)
            vd.set_geometry(col='geometry', inplace=True)
            vd.set_crs("EPSG:32640", allow_override=True, inplace=True)
            vd["THISSEN_LOCATION"] = vd.geometry.area / 1000000
            vd["THISSEN_AQUIFER"] = [mask_tmp.geometry.area[0] / 1000000] * len(gdf_tmp)
            vd["DATE_PERSIAN"] = [df["DATE_PERSIAN"].unique()[0]] * len(gdf_tmp)
            vd_result = vd_result.append(vd, ignore_index=True)                
    return vd_result

def f_interpolate (df_raw):
    df_raw = df_raw.reset_index(drop=True)
    df = df_raw.dropna(subset=["WATER_TABLE_INTERPOLATE"])
    vd_result = gpd.GeoDataFrame()    
    gdf_d = gdf[gdf["LOCATION_NAME"].isin(df["LOCATION_NAME"])]    
    # TODO: Find Better Way
    for i in range(len(mask)):
        mask_tmp = mask.iloc[[i]].reset_index(drop=True)
        gdf_d['CHECK'] = gdf_d['geometry'].apply(lambda x: mask_tmp.contains(x))
        gdf_tmp = gdf_d[gdf_d['CHECK']].reset_index(drop=True)        
        if len(gdf_tmp) > 0:
            vd = thiessen_polygons(gdf_tmp, mask_tmp)
            vd.set_geometry(col='geometry', inplace=True)
            vd.set_crs("EPSG:32640", allow_override=True, inplace=True)
            vd["THISSEN_LOCATION"] = vd.geometry.area / 1000000
            vd["THISSEN_AQUIFER"] = [mask_tmp.geometry.area[0] / 1000000] * len(gdf_tmp)
            vd["DATE_PERSIAN"] = [df["DATE_PERSIAN"].unique()[0]] * len(gdf_tmp)
            vd_result = vd_result.append(vd, ignore_index=True)                
    return vd_result

## CALCULATE

In [ ]:
# THISSEN ORIGINAL

vd_result = Data.groupby(
    by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN']
).apply(f)

vd_result = vd_result.droplevel(['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])

Data_Original = Data.merge(
    right=vd_result[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN', SELECTED_LEVEL, 'THISSEN_LOCATION', 'THISSEN_AQUIFER', 'geometry']],
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN']
).sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN']).rename(columns={SELECTED_LEVEL: 'LEVEL'})

Data_Original["THISSEN_LOCATION"] = np.where(Data_Original["THISSEN_LOCATION_x"].isna(), Data_Original["THISSEN_LOCATION_y"], Data_Original["THISSEN_LOCATION_x"])
Data_Original["THISSEN_AQUIFER"] = np.where(Data_Original["THISSEN_AQUIFER_x"].isna(), Data_Original["THISSEN_AQUIFER_y"], Data_Original["THISSEN_AQUIFER_x"])



# THISSEN INTERPOLATE

vd_result_interpolate = Data.groupby(
    by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN']
).apply(f_interpolate)

vd_result_interpolate = vd_result_interpolate.droplevel(['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])

Data_Interpolate = Data.merge(
    right=vd_result_interpolate[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN', SELECTED_LEVEL, 'THISSEN_LOCATION', 'THISSEN_AQUIFER', 'geometry']],
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_PERSIAN']
).sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN']).rename(columns={SELECTED_LEVEL: 'LEVEL'})

Data_Interpolate["THISSEN_LOCATION"] = np.where(Data_Interpolate["THISSEN_LOCATION_x"].isna(), Data_Interpolate["THISSEN_LOCATION_y"], Data_Interpolate["THISSEN_LOCATION_x"])
Data_Interpolate["THISSEN_AQUIFER"] = np.where(Data_Interpolate["THISSEN_AQUIFER_x"].isna(), Data_Interpolate["THISSEN_AQUIFER_y"], Data_Interpolate["THISSEN_AQUIFER_x"])

# Calculate Head Aquifer

## ORIGINAL

In [ ]:
# HEAD LOCATION
Data_Original['HEAD_LOCATION'] = Data_Original['LEVEL'] - Data_Original['WATER_TABLE']


# HEAD AQUIFER ARITHMETIC, GEOMETRIC, HARMONIC
tmp = Data_Original[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_LOCATION']].dropna(subset=['HEAD_LOCATION']).reset_index()\
    .groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .agg({'HEAD_LOCATION': [statistics.mean]})\
            .reset_index()

tmp.columns = [col for col in tmp.columns]
tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC']
# tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_GEOMETRIC', 'HEAD_AQUIFER_HARMONIC']

Data_Original = Data_Original.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])

del tmp


# HEAD AQUIFER THISSEN
Data_Original['TMP'] = (Data_Original['HEAD_LOCATION'] * Data_Original['THISSEN_LOCATION']) / Data_Original['THISSEN_AQUIFER']

tmp = Data_Original.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'HEAD_AQUIFER_THISSEN'})

# TODO: Find Better Way: Now, Replace All Zero with NaN
tmp['HEAD_AQUIFER_THISSEN'] = tmp['HEAD_AQUIFER_THISSEN'].replace({'0': np.nan, 0: np.nan})

Data_Original = Data_Original.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

In [ ]:
Data_Original

## INTERPOLATE

In [ ]:
# HEAD LOCATION
Data_Interpolate['HEAD_LOCATION_INTERPOLATE'] = Data_Interpolate['LEVEL'] - Data_Interpolate['WATER_TABLE_INTERPOLATE']


# HEAD AQUIFER ARITHMETIC, GEOMETRIC, HARMONIC
tmp = Data_Interpolate[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_LOCATION_INTERPOLATE']].dropna(subset=['HEAD_LOCATION_INTERPOLATE']).reset_index()\
    .groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .agg({'HEAD_LOCATION_INTERPOLATE': [statistics.mean]})\
            .reset_index()

tmp.columns = [col for col in tmp.columns]
tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC']
# tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_GEOMETRIC', 'HEAD_AQUIFER_HARMONIC']

Data_Interpolate = Data_Interpolate.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])

del tmp


# HEAD AQUIFER THISSEN
Data_Interpolate['TMP'] = (Data_Interpolate['HEAD_LOCATION_INTERPOLATE'] * Data_Interpolate['THISSEN_LOCATION']) / Data_Interpolate['THISSEN_AQUIFER']

tmp = Data_Interpolate.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'HEAD_AQUIFER_THISSEN'})

# TODO: Find Better Way: Now, Replace All Zero with NaN
tmp['HEAD_AQUIFER_THISSEN'] = tmp['HEAD_AQUIFER_THISSEN'].replace({'0': np.nan, 0: np.nan})

Data_Interpolate = Data_Interpolate.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

In [ ]:
Data_Interpolate

# Storage Coefficient

## ORIGINAL

In [ ]:
Data_Original['TMP'] = (Data_Original['STORAGE_COEFFICIENT_LOCATION'] * Data_Original['THISSEN_LOCATION']) / Data_Original['THISSEN_AQUIFER']

tmp = Data_Original.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'STORAGE_COEFFICIENT_AQUIFER'})

Data_Original = Data_Original.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

In [ ]:
Data_Original

## INTERPOLATE

In [ ]:
Data_Interpolate['TMP'] = (Data_Interpolate['STORAGE_COEFFICIENT_LOCATION'] * Data_Interpolate['THISSEN_LOCATION']) / Data_Interpolate['THISSEN_AQUIFER']

tmp = Data_Interpolate.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
            ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'STORAGE_COEFFICIENT_AQUIFER'})

Data_Interpolate = Data_Interpolate.merge(
    right=tmp,
    how='left',
    on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
            .drop(['TMP'], axis=1)

del tmp

In [ ]:
Data_Interpolate

In [1]:
# SAVE DATA
Data_Original.to_excel("Result/Data_Original.xlsx")
Data_Interpolate.to_excel("Result/Data_Interpolate.xlsx")

NameError: name 'Data_Original' is not defined

In [3]:
# LOAD DATA
xls = pd.ExcelFile('Result/Data_Original.xlsx')
Data_Original = pd.read_excel(xls, sheet_name='Sheet1')
Data_Original.DATE_GREGORIAN = Data_Original.DATE_GREGORIAN.apply(pd.to_datetime)
Data_Original.drop(['INDEX'], axis=1, inplace=True)


xls = pd.ExcelFile('Result/Data_Interpolate.xlsx')
Data_Interpolate = pd.read_excel(xls, sheet_name='Sheet1')
Data_Interpolate.DATE_GREGORIAN = Data_Interpolate.DATE_GREGORIAN.apply(pd.to_datetime)
Data_Interpolate.drop(['INDEX'], axis=1, inplace=True)

# Adjust

## Method 1: Delta

In [27]:
DATA_FIXED_DELTA_METHOD = Data_Original.groupby(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN"])["HEAD_AQUIFER_THISSEN"]\
    .mean().reset_index().sort_values(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN"])
    
DATA_FIXED_DELTA_METHOD = DATA_FIXED_DELTA_METHOD.dropna(subset=["HEAD_AQUIFER_THISSEN"]).reset_index()
    
threshold = 0.5

RESULT_FIXED_DELTA_METHOD = pd.DataFrame()

for mn in list(DATA_FIXED_DELTA_METHOD["MAHDOUDE_NAME"].unique()):
    tmp_mn = DATA_FIXED_DELTA_METHOD[(DATA_FIXED_DELTA_METHOD["MAHDOUDE_NAME"] == mn)]
    
    for an in list(tmp_mn["AQUIFER_NAME"].unique()):
        tmp_mn_an = tmp_mn[(tmp_mn["AQUIFER_NAME"] == an)]
        
        tmp_mn_an = tmp_mn_an.reset_index(drop=False)

        tmp_mn_an['Delta'] = tmp_mn_an['HEAD_AQUIFER_THISSEN'].diff().fillna(0)
        tmp_mn_an['Index'] = abs(tmp_mn_an['Delta']).apply(lambda x: 1 if x >= threshold else 0)
        tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN'] = tmp_mn_an['HEAD_AQUIFER_THISSEN']
        n = tmp_mn_an.index[tmp_mn_an['Index'] == True].tolist()

        if len(n) > 0:
            while len(n) != 0:
                delta = tmp_mn_an['Delta'][n[0]]
                tmp_mn_an['TEMP_HEAD_AQUIFER_THISSEN'] = tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN']
                for i in range(n[0]):
                    tmp_mn_an['TEMP_HEAD_AQUIFER_THISSEN'][i] = tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN'][i] + delta
                tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN'] = tmp_mn_an['TEMP_HEAD_AQUIFER_THISSEN']
                tmp_mn_an['Delta'] = tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN'].diff().fillna(0)
                tmp_mn_an['Index'] = abs(tmp_mn_an['Delta']).apply(lambda x: 1 if x >= threshold else 0)
                n = tmp_mn_an.index[tmp_mn_an['Index'] == True].tolist()

        if 'TEMP_HEAD_AQUIFER_THISSEN' in tmp_mn_an.columns:
            tmp_mn_an = tmp_mn_an.drop(['TEMP_HEAD_AQUIFER_THISSEN'], axis=1)

        if 'Delta' in tmp_mn_an.columns:
            tmp_mn_an = tmp_mn_an.drop(['Delta'], axis=1)

        if 'Index' in tmp_mn_an.columns:
            tmp_mn_an = tmp_mn_an.drop(['Index'], axis=1)
        
        RESULT_FIXED_DELTA_METHOD = pd.concat([RESULT_FIXED_DELTA_METHOD, tmp_mn_an], axis=0)
        
RESULT_FIXED_DELTA_METHOD

/tmp/ipykernel_420/3533254507.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,level_0,index,MAHDOUDE_NAME,AQUIFER_NAME,DATE_GREGORIAN,DATE_PERSIAN,HEAD_AQUIFER_THISSEN,ADJ_HEAD_AQUIFER_THISSEN
0,0,0,بردسکن,بردسکن,2001-10-07,1380-7-15,836.51,834.37
1,1,1,بردسکن,بردسکن,2001-11-06,1380-8-15,836.51,834.38
2,2,2,بردسکن,بردسکن,2001-12-06,1380-9-15,836.47,834.33
...,...,...,...,...,...,...,...,...
229,921,1177,گیسور,گیسور,2021-03-05,1399-12-15,846.31,840.06
230,922,1178,گیسور,گیسور,2021-04-04,1400-1-15,837.76,840.06
231,923,1179,گیسور,گیسور,2021-05-05,1400-2-15,840.06,840.06


## Method 2: Thissen

### Thissen: Test Location Thissen

In [28]:
data_2 = Data_Original.dropna(subset=["WATER_TABLE"])

CHECK_LOCATION = data_2.groupby(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN"])["LOCATION_NAME"]\
    .apply(list)\
        .reset_index(name='LOCATION_LIST').sort_values(['DATE_GREGORIAN'])


def f2(CHECK_LOCATION):
    result = CHECK_LOCATION.copy()
    result = result.reset_index(drop=True)
    CHECK_LOCATION_CONDI = []
    for i, l in enumerate(result.LOCATION_LIST):
        if i == 0:
            CHECK_LOCATION_CONDI.append(False)
        elif set(result.LOCATION_LIST[i]) == set(result.LOCATION_LIST[i-1]):
            CHECK_LOCATION_CONDI.append(False)
        else:
            CHECK_LOCATION_CONDI.append(True)
    result["CHECK_LOCATION_CONDI"] = CHECK_LOCATION_CONDI
    return result

CHECK_LOCATION_AQUIFERS = CHECK_LOCATION.groupby(by=["MAHDOUDE_NAME", "AQUIFER_NAME"])\
    .apply(f2).reset_index(drop=True)

data_2 = data_2.drop_duplicates(subset=["MAHDOUDE_NAME", "AQUIFER_NAME", 'DATE_GREGORIAN', 'DATE_PERSIAN'], keep='last').reset_index(drop=True)

data_2 = data_2.merge(
    right=CHECK_LOCATION_AQUIFERS,
    how='left',
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN"])\
        .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", 'DATE_GREGORIAN'])

data_2 = data_2[[
    "MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN",
    "HEAD_AQUIFER_THISSEN", "CHECK_LOCATION_CONDI",
]]

data_2

,MAHDOUDE_NAME,AQUIFER_NAME,DATE_GREGORIAN,DATE_PERSIAN,HEAD_AQUIFER_THISSEN,CHECK_LOCATION_CONDI
40,بردسکن,بردسکن,2001-10-07,1380-7-15,836.51,False
41,بردسکن,بردسکن,2001-11-06,1380-8-15,836.51,False
42,بردسکن,بردسکن,2001-12-06,1380-9-15,836.47,False
...,...,...,...,...,...,...
914,گیسور,گیسور,2021-03-05,1399-12-15,846.31,True
1144,گیسور,گیسور,2021-04-04,1400-1-15,837.76,True
1145,گیسور,گیسور,2021-05-05,1400-2-15,840.06,True


### Thissen: Adjust

In [29]:
DATA_THISSEN_METHOD = Data_Original.groupby(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN"])["HEAD_AQUIFER_THISSEN"]\
    .mean().reset_index().sort_values(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN"])
    
DATA_THISSEN_METHOD = DATA_THISSEN_METHOD.dropna(subset=["HEAD_AQUIFER_THISSEN"]).reset_index()   

RESULT_THISSEN_METHOD = pd.DataFrame()

for mn in list(DATA_THISSEN_METHOD["MAHDOUDE_NAME"].unique()):
    tmp_mn = DATA_THISSEN_METHOD[(DATA_THISSEN_METHOD["MAHDOUDE_NAME"] == mn)]
    data_2_mn = data_2[(data_2["MAHDOUDE_NAME"] == mn)]
    
    for an in list(tmp_mn["AQUIFER_NAME"].unique()):
        tmp_mn_an = tmp_mn[(tmp_mn["AQUIFER_NAME"] == an)]
        data_2_mn_an = data_2_mn[(data_2_mn["AQUIFER_NAME"] == an)]
        
        tmp_mn_an = tmp_mn_an.reset_index(drop=False)
        data_2_mn_an = data_2_mn_an.reset_index(drop=False)

        tmp_mn_an['Delta'] = tmp_mn_an['HEAD_AQUIFER_THISSEN'].diff().fillna(0)
        tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN'] = tmp_mn_an['HEAD_AQUIFER_THISSEN']
        
        n = data_2_mn_an[data_2_mn_an["CHECK_LOCATION_CONDI"]]["DATE_PERSIAN"].tolist()

        if len(n) > 0:
            
            for p_date in n:                
                delta = tmp_mn_an.loc[tmp_mn_an["DATE_PERSIAN"] == p_date, "Delta"].reset_index()["Delta"][0]
                ix = tmp_mn_an.loc[tmp_mn_an["DATE_PERSIAN"] == p_date, "Delta"].reset_index()["index"][0]
                tmp_mn_an['TEMP_HEAD_AQUIFER_THISSEN'] = tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN']
                for i in range(ix):                    
                    tmp_mn_an['TEMP_HEAD_AQUIFER_THISSEN'][i] = tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN'][i] + delta                    
                tmp_mn_an['ADJ_HEAD_AQUIFER_THISSEN'] = tmp_mn_an['TEMP_HEAD_AQUIFER_THISSEN']

        if 'TEMP_HEAD_AQUIFER_THISSEN' in tmp_mn_an.columns:
            tmp_mn_an = tmp_mn_an.drop(['TEMP_HEAD_AQUIFER_THISSEN'], axis=1)

        if 'Delta' in tmp_mn_an.columns:
            tmp_mn_an = tmp_mn_an.drop(['Delta'], axis=1)

        if 'Index' in tmp_mn_an.columns:
            tmp_mn_an = tmp_mn_an.drop(['Index'], axis=1)
        
        RESULT_THISSEN_METHOD = pd.concat([RESULT_THISSEN_METHOD, tmp_mn_an], axis=0)
        
RESULT_THISSEN_METHOD

/tmp/ipykernel_420/1571193954.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,level_0,index,MAHDOUDE_NAME,AQUIFER_NAME,DATE_GREGORIAN,DATE_PERSIAN,HEAD_AQUIFER_THISSEN,ADJ_HEAD_AQUIFER_THISSEN
0,0,0,بردسکن,بردسکن,2001-10-07,1380-7-15,836.51,825.99
1,1,1,بردسکن,بردسکن,2001-11-06,1380-8-15,836.51,826.00
2,2,2,بردسکن,بردسکن,2001-12-06,1380-9-15,836.47,825.95
...,...,...,...,...,...,...,...,...
229,921,1177,گیسور,گیسور,2021-03-05,1399-12-15,846.31,840.06
230,922,1178,گیسور,گیسور,2021-04-04,1400-1-15,837.76,840.06
231,923,1179,گیسور,گیسور,2021-05-05,1400-2-15,840.06,840.06


# Plot Aquifer Hydrograph

In [30]:
Data_Original_Aquifer = Data_Original.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).mean().reset_index()\
    .sort_values(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN"])[["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN", "HEAD_AQUIFER_ARITHMETIC", "HEAD_AQUIFER_THISSEN"]]

Data_Interpolate_Aquifer = Data_Interpolate.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).mean().reset_index()\
    .sort_values(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN"])[["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN", "HEAD_AQUIFER_ARITHMETIC", "HEAD_AQUIFER_THISSEN"]]
Data_Interpolate_Aquifer.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN", "HEAD_AQUIFER_ARITHMETIC_INTERPOLATE", "HEAD_AQUIFER_THISSEN_INTERPOLATE"]

RESULT_FIXED_DELTA_METHOD = RESULT_FIXED_DELTA_METHOD[["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN", "ADJ_HEAD_AQUIFER_THISSEN"]]
RESULT_FIXED_DELTA_METHOD.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN", "HEAD_AQUIFER_THISSEN_DELTA"]

RESULT_THISSEN_METHOD = RESULT_THISSEN_METHOD[["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN", "ADJ_HEAD_AQUIFER_THISSEN"]]
RESULT_THISSEN_METHOD.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN", "HEAD_AQUIFER_THISSEN_THISSEN"]
    
Data_Plot = Data_Original_Aquifer.merge(
    right=Data_Interpolate_Aquifer,
    how="outer",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN"]
).merge(
    right=RESULT_FIXED_DELTA_METHOD,
    how="outer",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN"]
).merge(
    right=RESULT_THISSEN_METHOD,
    how="outer",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "DATE_GREGORIAN", "DATE_PERSIAN"]
)

clc = data_2[data_2["CHECK_LOCATION_CONDI"]]

for mn in list(Data_Plot["MAHDOUDE_NAME"].unique()):
    Data_Plot_MN = Data_Plot[(Data_Plot["MAHDOUDE_NAME"] == mn)]
    clc_MN = clc[(clc["MAHDOUDE_NAME"] == mn)]
    
    for an in list(Data_Plot_MN["AQUIFER_NAME"].unique()):
        Data_Plot_AN = Data_Plot_MN[(Data_Plot_MN["AQUIFER_NAME"] == an)]
        clc_MN_AN = clc_MN[(clc_MN["AQUIFER_NAME"] == an)]
        
        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=Data_Plot_AN["DATE_PERSIAN"],
                y=Data_Plot_AN["HEAD_AQUIFER_THISSEN"],
                mode='lines+markers',
                name='HEAD_AQUIFER_THISSEN',
                marker=dict(
                    color='black',
                    size=12,
                ),
                line=dict(
                    color='black',
                    width=1
                )        
            )
        )
        
        # fig.add_trace(
        #     go.Scatter(
        #         x=Data_Plot_AN["DATE_PERSIAN"],
        #         y=Data_Plot_AN["HEAD_AQUIFER_ARITHMETIC"],
        #         mode='lines+markers',
        #         name='HEAD_AQUIFER_ARITHMETIC',
        #         marker=dict(
        #             color='red',
        #             size=12,
        #         ),
        #         line=dict(
        #             color='red',
        #             width=1
        #         )        
        #     )
        # )
        
        fig.add_trace(
            go.Scatter(
                x=Data_Plot_AN["DATE_PERSIAN"],
                y=Data_Plot_AN["HEAD_AQUIFER_THISSEN_INTERPOLATE"],
                mode='lines+markers',
                name='HEAD_AQUIFER_THISSEN_INTERPOLATE',
                marker=dict(
                    color='green',
                    size=12,
                ),
                line=dict(
                    color='green',
                    width=1
                )        
            )
        )
        
        # fig.add_trace(
        #     go.Scatter(
        #         x=Data_Plot_AN["DATE_PERSIAN"],
        #         y=Data_Plot_AN["HEAD_AQUIFER_ARITHMETIC_INTERPOLATE"],
        #         mode='lines+markers',
        #         name='HEAD_AQUIFER_ARITHMETIC_INTERPOLATE',
        #         marker=dict(
        #             color='yellow',
        #             size=12,
        #         ),
        #         line=dict(
        #             color='yellow',
        #             width=1
        #         )        
        #     )
        # )
        
        fig.add_trace(
            go.Scatter(
                x=Data_Plot_AN["DATE_PERSIAN"],
                y=Data_Plot_AN["HEAD_AQUIFER_THISSEN_THISSEN"],
                mode='lines+markers',
                name='HEAD_AQUIFER_THISSEN_THISSEN',
                marker=dict(
                    color='blue',
                    size=12,
                ),
                line=dict(
                    color='blue',
                    width=1
                )        
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=Data_Plot_AN["DATE_PERSIAN"],
                y=Data_Plot_AN["HEAD_AQUIFER_THISSEN_DELTA"],
                mode='lines+markers',
                name='HEAD_AQUIFER_THISSEN_DELTA',
                marker=dict(
                    color='yellow',
                    size=12,
                ),
                line=dict(
                    color='yellow',
                    width=1
                )        
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=clc_MN_AN["DATE_PERSIAN"],
                y=clc_MN_AN["HEAD_AQUIFER_THISSEN"],
                mode='markers',
                name='تغییر شبکه تیسن',
                marker=dict(
                    color='orange',
                    size=12,
                    symbol='x'
                )
            )
        )

        fig.update_layout(
            title_text=f"{Data_Plot_AN.MAHDOUDE_NAME.unique()[0]} --> {Data_Plot_AN.AQUIFER_NAME.unique()[0]}",
            autosize=False,
            width=1400,
            height=700,
            yaxis_title="تراز آبخوان - متر",
            font=dict(
                family="B Zar, Courier New, monospace",
                size=18,
                color="RebeccaPurple"
            ),
            xaxis=dict(
                tickformat="%Y-%m-%d"
            ),
        )

        fig.show()

# Plot Well Hydrograph

In [ ]:
for mn in list(Data_Original["MAHDOUDE_NAME"].unique()):
    for an in list(Data_Original["AQUIFER_NAME"].unique()):
        for ln in list(Data_Original["LOCATION_NAME"].unique()):
            
            df_original = Data_Original[(Data_Original["MAHDOUDE_NAME"] == mn) & (Data_Original["AQUIFER_NAME"] == an) & (Data_Original["LOCATION_NAME"] == ln)]
            df_interpolate = Data_Interpolate[(Data_Interpolate["MAHDOUDE_NAME"] == mn) & (Data_Interpolate["AQUIFER_NAME"] == an) & (Data_Interpolate["LOCATION_NAME"] == ln)]
            
            # Create Traces
            fig = go.Figure()
            
            fig.add_trace(
                go.Scatter(
                    x=df_original["DATE_PERSIAN"],
                    y=df_original["HEAD_LOCATION"],
                    mode='markers',
                    name='داده خام',
                    marker=dict(
                        color='black',
                        size=12,
                    )
                )
            )
            
            fig.add_trace(
                go.Scatter(
                    x=df_interpolate["DATE_PERSIAN"],
                    y=df_interpolate["HEAD_LOCATION_INTERPOLATE"],
                    mode='lines+markers',
                    name='داده شبیه سازی شده',
                    line=dict(
                        color='red',
                        width=1
                    ),
                    marker=dict(
                        color='red',
                        size=5,
                        opacity=1,
                    )
                ),
            )
            
            fig.update_layout(
                title=df_original["LOCATION_NAME"].unique()[0],
                yaxis_title="تراز سطح ایستابی",
                font=dict(
                    family="B Zar, Courier New",
                    size=18,
                    color="RebeccaPurple"
                ),
                xaxis=dict(
                    tickformat="%Y-%m-%d"
                ),
            )          
            
            fig.show()